In [6]:
from jupyter_plotly_dash import JupyterDash
import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "Un1cornp00p!"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    #Unique identifier
    html.H1('Raechel McGuire'),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label' :'Water Resue', 'value':1},
            {'label' :'Mountain Resue', 'value':2},
            {'label' :'Disaster Resue', 'value':3},
            {'label' :'Reset', 'value':4}
        ],
        value='MTL',
        labelStyle = {'display':'inline-block'}
    
    )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if (value =='Water Rescue'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'Mountain Rescue'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'Disaster Rescue'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('datatable-interactivity-container', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])

def update_graphs(viewData):
    derived_virtual_selected_rows = []
    dff = pd.DataFrame.from_dict(viewData)
    os.write(1,"TEST Pie\n".encode())
    os.write(1,dff.to_string().strip('[]').encode())

    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]

    return [
        dcc.Graph(            
            figure = px.pie(dff, values=dff.age_upon_outcome_in_weeks, names=dff.breed, color_discrete_sequence=px.colors.sequential.RdBu)
        )
        
    ]
@app.callback(
    Output('datatable-interactivity-map', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
    Input('datatable-interactivity', "derived_viewport_selected_rows")])
def update_map(viewData,rows):
    viewDF = pd.DataFrame.from_dict(viewData)
    
    dff = viewDF.loc[rows]
   
    
    # Austin TX is at [30.75,-97.48]
    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15, children=[
            dl.TileLayer(id="base-layer-id"),
    # Marker with tool tip and pop up
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                     html.H1("Animal Name"),
                     html.P(dff.loc[0,'name'])
                ])
            ])
    ])]


app